In [1]:
%pip install torchdata==0.5.1 --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install transformers==4.27.2
%pip install datasets==2.17.0 --quiet 

Note: you may need to restart the kernel to use updated packages.
  Using cached datasets-2.17.0-py3-none-any.whl.metadata (20 kB)
Using cached datasets-2.17.0-py3-none-any.whl (536 kB)
  Attempting uninstall: datasets
    Found existing installation: datasets 2.18.0
    Uninstalling datasets-2.18.0:
      Successfully uninstalled datasets-2.18.0
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install -U datasets

  Using cached datasets-2.18.0-py3-none-any.whl.metadata (20 kB)
Using cached datasets-2.18.0-py3-none-any.whl (510 kB)
  Attempting uninstall: datasets
    Found existing installation: datasets 2.17.0
    Uninstalling datasets-2.17.0:
      Successfully uninstalled datasets-2.17.0
Note: you may need to restart the kernel to use updated packages.


In [7]:
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

In [6]:
from datasets import load_dataset
huggingface_dataset_name = "knkarthick/dialogsum"
dataset = load_dataset(huggingface_dataset_name)

Generating train split: 12460 examples [00:00, 44393.90 examples/s]
Generating validation split: 500 examples [00:00, 20815.61 examples/s]
Generating test split: 1500 examples [00:00, 47058.27 examples/s]


## Summarize dialogue without prompt engineering

In [8]:
#This is the actual dataset no model is used 
example_indices = [40,200]
dash_line = '-'.join('' for x in range(100))

for i, index in enumerate(example_indices):
    print(dash_line)
    print('Example',i+1)
    print(dash_line)
    print('INPUT DIALOGUE:')
    print(dataset['test'][index]['dialogue'])
    print(dash_line)
    print('BASELINE HUMAN SUMMARY:')
    print(dataset['test'][index]['summary'])
    print(dash_line)
    print()

---------------------------------------------------------------------------------------------------
Example 1
---------------------------------------------------------------------------------------------------
INPUT DIALOGUE:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------

---------------------------------------------------------------------------------------------------
Exam

## loading the model

In [9]:
model_name = 'google/flan-t5-base'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

config.json: 100%|██████████| 1.40k/1.40k [00:00<00:00, 148kB/s]
c:\Users\vrinda\anaconda3\envs\gpu\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\vrinda\.cache\huggingface\hub\models--google--flan-t5-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
pytorch_model.bin: 100%|██████████| 990M/990

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_name,use_fast = True)

tokenizer_config.json: 100%|██████████| 2.54k/2.54k [00:00<00:00, 845kB/s]
spiece.model: 100%|██████████| 792k/792k [00:00<00:00, 1.21MB/s]
tokenizer.json: 100%|██████████| 2.42M/2.42M [00:01<00:00, 1.66MB/s]
special_tokens_map.json: 100%|██████████| 2.20k/2.20k [00:00<00:00, 275kB/s]


In [11]:
sentence = "What time is it,Tom?"
sentence_encoded = tokenizer(sentence, return_tensors ='pt')

sentence_decoded = tokenizer.decode(sentence_encoded['input_ids'][0],skip_special_tokens=True)

print('ENCODED SENTENCE:')
print(sentence_encoded['input_ids'][0])
print('\nDECODED SENTENCE')
print(sentence_decoded)

ENCODED SENTENCE:
tensor([ 363,   97,   19,   34,    6, 3696,   51,   58,    1])

DECODED SENTENCE
What time is it,Tom?


In [13]:
for i,index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary  = dataset['test'][index]['summary']
    inputs = tokenizer(dialogue,return_tensors ='pt')
    output = tokenizer.decode(model.generate(inputs['input_ids'],max_new_tokens =50,)[0],skip_special_tokens=True)

    print(dash_line)
    print('Example',i+1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{dialogue}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'Model generating output without prompt engineering:\n{output}')


---------------------------------------------------------------------------------------------------
Example 1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
Model generating output without prompt engineering:
Person1: It's ten to nine.
----------------------------

## SUMMARIZE DIALOGUE WITH AN INSTRUCTION PROMPT

## Zero Shot Inference with an Instruction Prompt

In [16]:
for i,index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary  = dataset['test'][index]['summary']

    prompt =f""" dialogue:.
    {dialogue}
    What was going on?
    """


    inputs = tokenizer(dialogue,return_tensors ='pt')
    output = tokenizer.decode(model.generate(inputs['input_ids'],max_new_tokens =50,)[0],skip_special_tokens=True)

    print(dash_line)
    print('Example',i+1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{dialogue}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'Model generating output using zero shot:\n{output}')


---------------------------------------------------------------------------------------------------
Example 1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
Model generating output using zero shot:
Person1: It's ten to nine.
---------------------------------------

## Summarize Dialogue with One shot and Few shot Inference

## ONE SHOT INFERENCE

In [17]:
def make_prompt(example_indices_full,example_index_to_summarize):
    prompt =''
    for index in example_indices_full:
        dialogue = dataset['test'][index]['dialogue']
        summary = dataset['test'][index]['summary']

        # the stop sequence "{summary}\n\n\n" is important for FLAN-t5.Other models may have their own preferred stop sequence.
        prompt += f"""
        Dialogue:
        {dialogue}
        what was going on?
        {summary}

        """
        dialogue = dataset['test'][example_index_to_summarize]['dialogue']
        prompt +=f"""
Dialogue:
{dialogue}
what was going on?
"""
    return prompt 

## Constructing the prompt to perform one shot inference

In [18]:
example_indices_full = [40]
example_index_to_summarize = 200

one_shot_prompt = make_prompt(example_indices_full,example_index_to_summarize)

In [19]:
print(one_shot_prompt)


        Dialogue:
        #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
        what was going on?
        #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.

        
Dialogue:
#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster proce

In [20]:
summary  = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(one_shot_prompt,return_tensors ='pt')
output = tokenizer.decode(model.generate(inputs['input_ids'],max_new_tokens =50,)[0],skip_special_tokens=True)


print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}')
print(dash_line)
print(f'Model generating output using one shot:\n{output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
---------------------------------------------------------------------------------------------------
Model generating output using one shot:
#Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to add a CD-ROM drive.


## FEW SHOT INFERENCE

In [21]:
example_indices_full = [40,80,120]
example_index_to_summarize = 200

few_shot_prompt = make_prompt(example_indices_full,example_index_to_summarize)
print(few_shot_prompt)


        Dialogue:
        #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
        what was going on?
        #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.

        
Dialogue:
#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster proce

In [23]:
summary  = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(few_shot_prompt,return_tensors ='pt')
output = tokenizer.decode(model.generate(inputs['input_ids'],max_new_tokens =50,)[0],skip_special_tokens=True)


print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}')
print(dash_line)
print(f'Model generating output using few shot:\n{output}')

Token indices sequence length is longer than the specified maximum sequence length for this model (1275 > 512). Running this sequence through the model will result in indexing errors


---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
---------------------------------------------------------------------------------------------------
Model generating output using few shot:
#Person1 wants to upgrade his computer. #Person2 wants to upgrade his computer.


## CONFIGURATION PARAMETERS

In [24]:
Generation_Config = GenerationConfig(max_new_tokens = 50)
# Generation_Config = GenerationConfig(max_new_tokens = 50,do_sample =True,temperature= 0.1)
inputs = tokenizer(few_shot_prompt,return_tensors ='pt')
output = tokenizer.decode(model.generate(inputs['input_ids'],max_new_tokens =50,)[0],skip_special_tokens=True)


print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}')
print(dash_line)
print(f'Model generating output using few shot:\n{output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
---------------------------------------------------------------------------------------------------
Model generating output using few shot:
#Person1 wants to upgrade his computer. #Person2 wants to upgrade his computer.
